In [1]:
%reload_ext autoreload
%autoreload 2
## sys package
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"  # specify which GPU(s) to be used
sys.path.append("./prediction_models/input/prostate-cancer-grade-assessment/")
sys.path.append("./input/")
## warning off
import warnings
warnings.filterwarnings("ignore")

## general package
import random
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
from torch.utils.data import *
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

## customized package
from input.inputPipeline_spine_infer_dataspecific import PandaPatchDatasetInfer
from model.resnext_ssl_stiching import Model_Infer as Model

### Define test folder

In [5]:
DATA = './input/prostate-cancer-grade-assessment/test_images'
TEST = './input/prostate-cancer-grade-assessment/test.csv'
TRAIN = './input/prostate-cancer-grade-assessment/train.csv'
TRAIN_rad = './input/prostate-cancer-grade-assessment/train.csv'
SAMPLE = './input/prostate-cancer-grade-assessment/sample_submission.csv'
# is_test = os.path.exists(DATA)
is_test = False

### Define Model

In [33]:
models_r = []
models_k = []
weights_r = [f'./train/weights/Resnext50_36patch_adam_cos_spine_rad/Resnext50_36patch_adam_cos_spine_rad_{i}_best.pth.tar' for i in [0,3]]
weights_k = [f'./train/weights/Resnext50_medreso_36patch_adam_cosine_bin/Resnext50_medreso_36patch_adam_cosine_bin_{i}_best.pth.tar' for i in [2,3]]
for path in weights_r:
    state_dict = torch.load(path)
    model = Model()
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.cuda()
    models_r.append(model)
for path in weights_k:
    state_dict = torch.load(path)
    model = Model()
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.cuda()
    models_k.append(model)

del state_dict

### Inference

In [7]:
tsfm = None
sub_df = pd.read_csv(SAMPLE)
## if there are data in "test_images", (only happens when you submit your notebook, do inference)

sz = 256
bs = 1

if is_test:
    dataset0 = PandaPatchDatasetInfer(TEST, DATA, sz, transform=tsfm, N = 36, rand = False)
#     dataset1 = PandaPatchDatasetInfer(TEST, DATA, sz, transform=tsfm, N = 36, rand = False, mode = 2)
else:
    dataset0 = PandaPatchDatasetInfer(TRAIN, DATA_TRAIN, sz, transform=tsfm, N = 36, rand = False)
#     dataset1 = PandaPatchDatasetInfer(TRAIN, DATA_TRAIN, sz, transform=tsfm, N = 36, rand = False, mode = 2)
dataloader0 = DataLoader(dataset0, batch_size=bs,
                        shuffle=False, num_workers=4)
# dataloader1 = DataLoader(dataset1, batch_size=bs,
#                         shuffle=False, num_workers=4)

In [35]:
names = [] ## record image names and predictions
preds = {}
for i in range(1 * len(models_k)):
    preds[f'{i}'] = []

In [36]:
## Model inference
with torch.no_grad():
    for idx, data in enumerate(tqdm(dataloader0)):
        img, name, provider = data['img'], data['name'], data['datacenter']
        img = img.cuda()
        if provider[0] == "radboud":
            for i,model in enumerate(models_r):
                logits = model(img)['out']
                preds[f'{i}'].append(logits)
            names.append(name)
        else:
            for i,model in enumerate(models_k):
                logits = model(img)['out']
                preds[f'{i}'].append(logits)
            names.append(name)        
        
        if not is_test:
            if idx >= 400:
                break
      
    names = np.concatenate(names)
    predictions = 0
    for i in range(1 * len(models_r)):
        predictions += torch.cat(preds[f'{i}']).sigmoid().cpu()
    predictions = predictions / (1 * len(models_r))
    predictions = predictions.sum(1).round().numpy().astype(int)
#     print("preds", preds.shape)
    sub_df = pd.DataFrame({'image_id': names, 'isup_grade': predictions})

sub_df.to_csv('submission.csv', index=False)
sub_df.head(100)

,image_id,isup_grade
0,0005f7aaab2800f6170c399693a96917,0
1,000920ad0b612851f8e01bcc880d9b3d,0
2,0018ae58b01bdadc8e347995b69f99aa,4
3,001c62abd11fa4b57bf7a6c603a11bb9,4
4,001d865e65ef5d2579c190a0e0350d8f,0
...,...,...
95,0277b937ddc8679ae69b3c54bfd714f1,5
96,027c4a5b5f9f1be80602ef51f2619bc2,1
97,02805e3e20ef629ffeeee1e68cf24ab0,4
98,028098c36eb49a8c6aa6e76e365dd055,0


In [22]:
import pandas as pd
TRAIN = './input/prostate-cancer-grade-assessment/train.csv'
df = pd.read_csv(TRAIN)

In [38]:
preds = sub_df.loc[:400,'isup_grade'].tolist()
label = df.loc[:400,'isup_grade'].tolist()
wrong = 0
for (i,j) in zip(preds,label):
    if i != j:
        wrong += 1
print(f'total wrong predictions {wrong}, percent {wrong / len(label)}')

total wrong predictions 39, percent 0.09725685785536159


In [31]:
kappa = cohen_kappa_score(label, preds, weights='quadratic')
print("The average kappa score is {}.".format(kappa))

The average kappa score is 0.9826394857162501.
